<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/schlager-lyrics-bot/blob/main/Schlage_bot_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.3
!pip install datasets
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install trl
!pip install safetensors
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
#Check CUDA is running on GPU and bf16 supported (required)

import torch
print(torch.cuda.is_available(), torch.cuda.is_bf16_supported())

True False


In [3]:
#Import dependencies

from random import randrange
import pandas as pd
from datasets import Dataset

In [4]:
df_lyrics = pd.read_csv('schlager_songs_v2.csv')

df_lyrics['lyrics'][10]

'{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}'

In [5]:
# Create separate column for 1st verse

verse_list = []

for ind in df_lyrics["lyrics"].index:
    lyrics = df_lyrics["lyrics"].iloc[ind]
    if lyrics == '{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}' :
        verse_list.append(None)
    else:
        try:
            verse = lyrics.split("\n")[0]
            n = 1
            while len(' '.join(verse).split()) <= 15:
                verse = lyrics.split("\n")[0:n]
                n += 1
            verse_list.append(''.join(verse))

        except:
            verse_list.append(None)


df_lyrics["verse_1"] = verse_list

df_lyrics["verse_1"][0]

df_lyrics.to_csv("schlager_songs_v2.csv")

In [6]:
# Prompt Example

### Instruction:

"Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben."

### Input:

"Du sagst nicht ein Wort"

### Response:

f""" Du sagst nicht ein Wort
 Und deine Hand wischt eine Träne fort
 Und dein leerer Blick
 Sinkt in dein Glas
 Du sitzt hier vor mir
 Und dein Gesicht lässt keinen Zweifel mehr
 Heut sagst du mir
 Dass ich dich verlier'
 Nie war Zeit für dich
 Ich lebte nur in meiner eignen Welt
 Ich weiß, du wirst gehen
 Ich muss dich versteh'n
 Lieb mich ein letztes mal
 Es bleibt mir keine andre Wahl
 Ich weiß, dass ich die Nacht mit dir
 An den Tag verlier'
...
 Und bleib bei mir"""

" Du sagst nicht ein Wort\n Und deine Hand wischt eine Träne fort\n Und dein leerer Blick\n Sinkt in dein Glas\n Du sitzt hier vor mir\n Und dein Gesicht lässt keinen Zweifel mehr\n Heut sagst du mir\n Dass ich dich verlier'\n Nie war Zeit für dich\n Ich lebte nur in meiner eignen Welt\n Ich weiß, du wirst gehen\n Ich muss dich versteh'n\n Lieb mich ein letztes mal\n Es bleibt mir keine andre Wahl\n Ich weiß, dass ich die Nacht mit dir\n An den Tag verlier'\n...\n Und bleib bei mir"

In [7]:
def format_instruction(sample):

    output_texts = []

    for i in range(len(sample['input'])):
      text = f"""### Anweisung:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

{sample['input']}

### Output:

{sample['output']}"""

      output_texts.append(text)

    return output_texts


In [8]:
#Create input and output dataset from dataframe

df_dataset = df_lyrics[["verse_1", "lyrics"]]

ind_nan = []

for ind in df_dataset.index:
    if df_dataset["verse_1"].iloc[ind] == None:
        ind_nan.append(ind)
    else:
        continue

df_dataset.drop(index=ind_nan, inplace=True)

df_dataset.reset_index(drop=True, inplace=True)

df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)

<ipython-input-8-933d3ca1faf6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.drop(index=ind_nan, inplace=True)
<ipython-input-8-933d3ca1faf6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)


In [9]:
print(len(df_dataset))

1046


In [10]:
train_df = df_dataset[:946]
test_df = df_dataset[946:]

In [11]:
from datasets.dataset_dict import DatasetDict

datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
    })

datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 946
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 100
    })
})

In [12]:
print(format_instruction(datasets_train_test["train"][randrange(946)])[0])

### Anweisung:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

 Ich ging meinen Weg durch die Stille der Nacht und hab'Dich gefunden

### Output:

 Ich ging meinen Weg durch die Stille der Nacht und hab'Dich gefunden
 Wir wussten es beide, das wird soviel mehr, als ein paar schöne Stunden
 Nein, diesmal zählt alles - die Hoffnung, die Tränen, das Glück,
 Ich hab'Deine Liebe und geb Dir auch meine nicht nur im Augenblick
 Im Kartenhaus der Träume, da gibt es Schatten und Licht
 Ein Spiel mit tausend Farben, wo jeder was andres verspricht
 Ich will Dich nie verlieren, unsre Liebe ist bestimmt
 So Viel mehr als nur ein Kartenhaus im Wind
 Vielleicht sind wir Träumer - zu viel Gefühl, wie die anderen sagen
 Doch was sie auch denken, es ist unser Spiel, es sind unsere Fragen
 Und wenn es passiert, das der Sturm ein paar Träume verweht,
 Dann bau'n wir sie wieder neu auf, denn ich glaube es ist, niemals zu spät
 Im Kartenhaus der Träume, da gibt es Schatt

*Break Start*

In [13]:
# Import dependencies

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, default_data_collator, get_linear_schedule_with_warmup
from trl import SFTTrainer
import accelerate
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
#SFTTrainer supports a native integration with peft, which makes it easy to instruction tune LLMs

from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType


In [14]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
#Parameters
device = "cuda"
model_id = "LeoLM/leo-hessianai-7b" # non-gated

peft_config = PromptTuningConfig(
            task_type="CAUSAL_LM",
            prompt_tuning_init=PromptTuningInit.TEXT,
            num_virtual_tokens=8,
            prompt_tuning_init_text="Benuzte den gegebenen Input um ein Schlager Lied zu schreiben.",
            tokenizer_name_or_path=model_id
)

dataset_name = "schlager_lyrics"
checkpoint_name = f"{dataset_name}_{model_id}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "input"
label_column = "output"
max_length = 300 #optimize
lr = 6e-2
num_epochs = 30
batch_size = 3


Preprocess dataset

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [27]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{x}" for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(datasets_train_test["train"][0:10])

In [29]:
processed_datasets = datasets_train_test.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=datasets_train_test["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [30]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True)

Train

In [31]:
model = AutoModelForCausalLM.from_pretrained(model_id)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 32,768 || all params: 6,738,448,384 || trainable%: 0.0004862840543203603
None


In [32]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [33]:
#Maximising the space available for model loading on GPU
os.environ['PYTORCH_CUDA_ALLOC_CONF']="max_split_size_mb:1024"


In [34]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True))

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

OutOfMemoryError: ignored

Notes:

v01:

- Epoch 21: Loss is levelling out
- Training Loss of last epoch (50) = 2.1387

Inference Test: 5.5/10

v02:

- optimise GPU memory allocation to make use of unallocated space
- parameters:
  - max_length = 500
  - lr = 2e-4
  - num_epochs = 50
  - batch_size = 5

- Training Loss of last epoch (50) = 1.5997
- Evaluation Loss of last epoch (50) = 1.5925

Inference Test: 4/10

v03:

- changed prompt to: "Benuzte den gegebenen Input um ein Schlager Lied zu schreiben." (previously: "Benuzte den gegebenen Vers um den Text fur ein Schlager Lied zu schreiben."

- parameters:
  - max_length = 300
  - lr = 3e-2
  - num_epochs = 30
  - batch_size = 6

- Training Loss of last epoch (30) = 1.8726
- Evaluation Loss of last epoch (30) = 1.9066

Inference Test: 6.5/10

v04

- parameters:
  - max_length = 500
  - lr = 6e-2
  - num_epochs = 30
  - batch_size = 5

- Training Loss of last epoch (30) = 1.5883
- Evaluation Loss of last epoch (30) = 1.5758

Inference Test: 7.5/10

v05

- change base model to:
malteos/bloom-6b4-clp-german

- parameters:
  - max_length = 500
  - lr = 8e-2
  - num_epochs = 30
  - batch_size = 5

- Training Loss of last epoch (30) =
- Evaluation Loss of last epoch (30) =

Inference Test: /10

*Share model to huggingface hub*

In [35]:
peft_model_id = "niclasfw/bloom-1b5-schlager-bot-004"
model.push_to_hub(peft_model_id, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:853: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/131k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/niclasfw/bloom-1b5-schlager-bot-004/commit/d1267560271e8808be6037ecb88bd5b510a75fff', commit_message='Upload model', commit_description='', oid='d1267560271e8808be6037ecb88bd5b510a75fff', pr_url=None, pr_revision=None, pr_num=None)

*Inference*

In [ ]:
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)


In [52]:
model = model.to(device)
sample = f"""Wir feiern das Leben laut\n
Wir feiern das Leben laut\n
Wir stellen keine Fragen, das Leben wird uns tragen\n
"""

prompt = f"""### Instruction:
          Benuzte den gegebenen Input um ein Schlager Lied zu schreiben.

### Input:
{sample}

"""

for n in range(5):
        input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
        # with torch.inference_mode():
        outputs = model.generate(input_ids=input_ids, pad_token_id=tokenizer.eos_token_id, max_new_tokens=300, do_sample=True, top_p=0.9,temperature=0.9)

        print(f"Version {n+1}")
        print(f"Prompt:\n{sample}\n")
        print(f"Generated output:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")

Version 1
Prompt:
Wir feiern das Leben laut

Wir feiern das Leben laut

Wir stellen keine Fragen, das Leben wird uns tragen

Ohne Zweifel alles wagen

Den höchsten Berg besteigen, die ganze Welt bereisen

Du und ich für alle Zeiten

            

Generated output:
### Input:
Wir feiern das Leben laut
 Wir feiern das Leben laut
 Wir stellen keine Fragen, das Leben wird uns tragen
 Ohne Zweifel alles wagen
 Wir leben in der Welt
 Alles ist möglich
 Es gibt keine Grenzen
 Alles ist erlaubt
 Alles ist erlaubt

 Alles ist möglich
 Es gibt keine Grenzen
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles ist erlaubt
 Alles is

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=peft_model_id)

In [ ]:
outputs = generator("Wo war ich in der Nacht von Freitag auf Montag? War ich drei, drei Tage wach oder einfach im Koma?",
                     num_return_sequences=1, max_length=500)
outputs

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: ignored

In [ ]:
#encode context the generation is conditioned on
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

In [ ]:
#Test input
inputs = tokenizer(
    f'{input} : {"Wo war ich in der Nacht von Freitag auf Montag? War ich drei, drei Tage wach oder einfach im Koma?"} output :',
    return_tensors="pt",
)

In [ ]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        inputs=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:3 for open-end generation.


['<bound method Kernel.raw_input of <google.colab._kernel.Kernel object at 0x78d097e5e2c0>> : Wo war ich in der Nacht von Freitag auf Montag? War ich drei, drei Tage wach oder einfach im Koma? output : <google.colab._kernel.']


*Break End*

In [ ]:
# Hugging Face model id
model_id = "malteos/bloom-6b4-clp-german" # non-gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


bloom_model = BloomModelForCausalLM.from_pretrained(
      model_id,
      load_in_8bit=True,
      # quantization_config=bnb_config,
      use_cache=False,
      torch_dtype=torch.float16,
      device_map="auto")

# model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM"
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
#We need to define the hyperparameters before we can start training

args = TrainingArguments(
    output_dir="./content/drive/MyDrive/AiCore/trained_model/schlager-bot-001",
    num_train_epochs=3,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    # bf16=True,
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    # disable_tqdm=True  # disable tqdm since with packing values are in correct
)

In [ ]:
#We can now configure the trainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    # peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

Map:   0%|          | 0/1046 [00:00<?, ? examples/s]

In [ ]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()